In [31]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity
from implicit.nearest_neighbours import CosineRecommender
# from implicit.recommender_base import RecommenderBase 
interactions_df = pd.read_csv('data/interactions.csv')
items_df = pd.read_csv('data/items.csv')
users_df = pd.read_csv('data/users.csv')

In [32]:
class MyCosineRecommender:
    def __init__(self):
        self.user_factors = None
        self.item_factors = None
    
    def fit(self, user_item_matrix):
        """
        Тренирует модель на основе разреженной матрицы предпочтений.
        :param user_item_matrix: csr_matrix, где строки — пользователи, столбцы — элементы.
        """
        # Косинусное сходство для элементов (матрица item-item)
        self.item_factors = cosine_similarity(user_item_matrix.T, dense_output=False)

    def recommend(self, user_id, user_item_matrix, N=10):
        """
        Генерирует топ-N рекомендаций для пользователя на основе косинусного сходства.
        :param user_id: ID пользователя
        :param user_item_matrix: csr_matrix с предпочтениями пользователя
        :param N: количество рекомендаций
        :return: список рекомендованных элементов
        """
        user_interactions = user_item_matrix[user_id]
        scores = user_interactions.dot(self.item_factors)

        recommended_items = np.argsort(scores.data)[::-1][:N]

        return recommended_items, scores.data



In [33]:
interactions = np.random.binomial(1, 0.1, size=(100, 1000))

In [34]:
cos_recommender = CosineRecommender()
cos_recommender.fit(interactions)

/home/umbilnm/python_ml/venvs/graphs2/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0001785755157470703 seconds
  warnings.warn(
100%|██████████| 1000/1000 [00:00<00:00, 56494.27it/s]


## Implicit Cosine Recommender

In [37]:
users_inv_mapping = dict(enumerate(interactions_df['user_id'].unique()))
users_mapping = {v: k for k, v in users_inv_mapping.items()}

items_inv_mapping = dict(enumerate(interactions_df['item_id'].unique()))
items_mapping = {v: k for k, v in items_inv_mapping.items()}

def generate_implicit_recs_mapper(
    model: ItemItemRecommender,
    train_matrix: csr_matrix,
    top_N: int,
    user_mapping: dict,
    item_inv_mapping: dict,
    filter_already_liked_items: bool
):
    def _recs_mapper(user: int):
        user_id = user_mapping[user]
        recs = model.recommend(user_id, 
                               train_matrix, 
                               N=top_N, 
                               filter_already_liked_items=filter_already_liked_items)
        return [item_inv_mapping[item] for item, _ in recs]
    return _recs_mapper